# Q-Learning

In [36]:
import matplotlib.pyplot as plt
import os
import random
import sys

cwd = os.getcwd()

parentdir = os.path.dirname(cwd)
sys.path.insert(0, parentdir)

from environments.tic_tac_toe.tic_tac_toe import TicTacToe
from environments.game import PlayRoom

In [37]:
playroom = PlayRoom(game = TicTacToe())

class Transistion():
  def __init__(self, state_code: str, action: int, reward: int, next_state_code: str):
    self.state_code = state_code
    self.action = action
    self.next_state_code = next_state_code
    self.reward = reward

In [38]:
##EPSILON STILL NEEDED

class QLearning():
  def __init__(self, learning_rate: float, discount_factor: float, epsilon: float):
    self.Q_table = {}
    self.playroom = PlayRoom(game = TicTacToe())
    self.epsilon = epsilon
    self.learning_rate = learning_rate
    self.discount_factor = discount_factor

  def determine_discount_factor(self):
    for df in range(0.1, 1.0, 0.1):
      self.discount_factor = df
      self.reset()
      eval_x, eval_y = self.learn(n_epochs = 20_000, eval_freq = 1_000, num_eval_episodes = 500)
      plt.show(eval_x, eval_y, label = f'{df}')
    
    plt.legend()
    plt.title('Peformance during training for different discount factors')
    plt.ylabel('Trained epochs')
    plt.xlabel('Average evaluation reward')
    plt.show()
    
  def determine_epsilon(self):
    for eps in range(0.1, 1.0, 0.1):
      self.epsilon = eps
      self.reset()
      eval_x, eval_y = self.learn(n_epochs = 20_000, eval_freq = 1_000, num_eval_episodes = 500)
      plt.show(eval_x, eval_y, label = f'{eps}')
    
    plt.legend()
    plt.title('Peformance during training for different epsilons')
    plt.ylabel('Trained epochs')
    plt.xlabel('Average evaluation reward')
    plt.show()

  def determine_learning_rate(self):
    for lr in range(0.1, 1.0, 0.1):
      self.learning_rate = lr
      self.reset()
      eval_x, eval_y = self.learn(n_epochs = 20_000, eval_freq = 1_000, num_eval_episodes = 500)
      plt.show(eval_x, eval_y, label = f'{lr}')
    
    plt.legend()
    plt.title('Peformance during training for different learning rates')
    plt.ylabel('Trained epochs')
    plt.xlabel('Average evaluation reward')
    plt.show()

  def reset(self):
    self.Q_table = {}

  def learn(self, n_epochs: int, eval_freq: int, num_eval_episodes: int, early_stop: bool = True, early_stop_num: int = 10):
    self.playroom.reset()
    done = False

    eval_y = []
    eval_x = []
    highest_eval = 0
    not_improved = 0

    for epoch in range(n_epochs):
      action = self._determine_action(deterministic = False)
      state_action_code = f'{self.playroom.game_state_code}-{action}'

      if state_action_code not in self.Q_table.keys():
        self.Q_table[state_action_code] = 0

      _, reward, done, _, _ = self.playroom.step(action = action)
      
      if done:
        self.playroom.reset()
        self.Q_table[state_action_code] = (1 - self.learning_rate) * self.Q_table[state_action_code] + self.learning_rate * (reward)
      
      else:
        next_optimal_state_action_code = f'{self.playroom.game_state_code}-{self._determine_action(deterministic = True)}'

        if next_optimal_state_action_code not in self.Q_table.keys():
          self.Q_table[next_optimal_state_action_code] = 0

        self.Q_table[state_action_code] = (1 - self.learning_rate) * self.Q_table[state_action_code] + self.learning_rate * (reward + self.discount_factor * self.Q_table[next_optimal_state_action_code])

      if (epoch + 1) % eval_freq == 0:
        eval_y.append(self._evaluate(num_eval_episodes = num_eval_episodes))
        eval_x.append(epoch + 1)

        if highest_eval < eval_y[-1]:
          highest_eval = eval_y[-1]
          not_improved = 0
        else:
          not_improved += 1

        print(f'Evaluate {eval_x[-1]}/{n_epochs}: {eval_y[-1]}')
        self.playroom.reset()

        if early_stop and not_improved > early_stop_num:
          break
    
    plt.plot(eval_x, eval_y)
    plt.title('Average reward during evalation')
    plt.xlabel("Trained number of epochs")
    plt.ylabel('Average reward')
    plt.show()

    return eval_x, eval_y


  def _determine_action(self, deterministic):
    if random.random() < self.epsilon and not deterministic:
      return random.choice(self.playroom.possible_actions)
    
    state_code = self.playroom.game_state_code
    state_q_values = {k: v for k, v in self.Q_table.items() if state_code in k}

    if len(state_q_values) == 0:
      return random.choice(self.playroom.possible_actions)
    
    max_state_action = max(state_q_values, key = state_q_values.get)
    return int(max_state_action.split('-')[-1])
  
  def _evaluate(self, num_eval_episodes: int):
    total_reward = 0
    for _ in range(num_eval_episodes):
      self.playroom.reset()
      done = False
      while not done:
        _, reward, done, _, _ = self.playroom.step(action = self._determine_action(deterministic = True))
      total_reward += reward
    return total_reward / num_eval_episodes

In [39]:
q_learing = QLearning(learning_rate = 0.9, discount_factor = 0.9, epsilon = 0.5)
q_learing.learn(200_000, eval_freq = 1_000, num_eval_episodes = 500)

Evaluate 1000/200000: -0.184
Evaluate 2000/200000: -0.084
Evaluate 3000/200000: -0.236
Evaluate 4000/200000: -0.088
Evaluate 5000/200000: -0.132
Evaluate 6000/200000: -0.024
Evaluate 7000/200000: -0.084
Evaluate 8000/200000: 0.064
Evaluate 9000/200000: 0.148
Evaluate 10000/200000: 0.02
Evaluate 11000/200000: 0.052
Evaluate 12000/200000: -0.056
Evaluate 13000/200000: 0.072
Evaluate 14000/200000: 0.12
Evaluate 15000/200000: 0.144
Evaluate 16000/200000: 0.248
Evaluate 17000/200000: 0.256
Evaluate 18000/200000: 0.216
Evaluate 19000/200000: 0.284
Evaluate 20000/200000: 0.176
Evaluate 21000/200000: 0.3
Evaluate 22000/200000: 0.22
Evaluate 23000/200000: 0.22


KeyboardInterrupt: 